In [2]:
import gmaps
import time
import requests
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plot
from config import weather_api_key
from config import g_key
from datetime import datetime
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [3]:
city_data_df=pd.read_csv("../weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 79


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
7,7,Touros,BR,2020-10-21 19:27:37,-5.20,-35.46,77.00,94,75,11.41
19,19,Saint-Joseph,RE,2020-10-21 19:32:07,-21.37,55.62,75.20,69,23,16.11
20,20,Dingle,PH,2020-10-21 19:29:49,11.00,122.67,76.32,91,100,6.82
30,30,Victoria,HK,2020-10-21 19:27:25,22.29,114.16,75.00,66,100,1.01
31,31,Bongandanga,CD,2020-10-21 19:32:13,1.50,21.05,70.34,97,100,1.41
37,37,Kutum,SD,2020-10-21 19:32:16,14.20,24.67,77.76,16,0,12.15
39,39,Conde,BR,2020-10-21 19:32:17,-7.26,-34.91,78.80,88,75,18.34
44,44,Souillac,MU,2020-10-21 19:32:19,-20.52,57.52,75.20,73,40,10.29
50,50,Capoeiras,BR,2020-10-21 19:32:22,-8.73,-36.63,74.57,64,26,13.15
51,51,Cap Malheureux,MU,2020-10-21 19:32:22,-19.98,57.61,75.00,72,55,7.00


In [10]:
preferred_cities_df.count()

City_ID       136
City          136
Country       134
Date          136
Lat           136
Lng           136
Max Temp      136
Humidity      136
Cloudiness    136
Wind Speed    136
dtype: int64

In [11]:
preferred_cities_df=preferred_cities_df.dropna()

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,
19,Saint-Joseph,RE,75.20,-21.37,55.62,
20,Dingle,PH,76.32,11.00,122.67,
30,Victoria,HK,75.00,22.29,114.16,
31,Bongandanga,CD,70.34,1.50,21.05,
37,Kutum,SD,77.76,14.20,24.67,
39,Conde,BR,78.80,-7.26,-34.91,
44,Souillac,MU,75.20,-20.52,57.52,
50,Capoeiras,BR,74.57,-8.73,-36.63,
51,Cap Malheureux,MU,75.00,-19.98,57.61,


In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,
19,Saint-Joseph,RE,75.20,-21.37,55.62,
20,Dingle,PH,76.32,11.00,122.67,
30,Victoria,HK,75.00,22.29,114.16,
31,Bongandanga,CD,70.34,1.50,21.05,
37,Kutum,SD,77.76,14.20,24.67,
39,Conde,BR,78.80,-7.26,-34.91,
44,Souillac,MU,75.20,-20.52,57.52,
50,Capoeiras,BR,74.57,-8.73,-36.63,
51,Cap Malheureux,MU,75.00,-19.98,57.61,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [16]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,INN NEW HORIZON
19,Saint-Joseph,RE,75.20,-21.37,55.62,"""Plantation Bed and Breakfast"
20,Dingle,PH,76.32,11.00,122.67,Camp Pasica
30,Victoria,HK,75.00,22.29,114.16,Mini Hotel Central
31,Bongandanga,CD,70.34,1.50,21.05,
37,Kutum,SD,77.76,14.20,24.67,Dibbo areaمنطقة دبو
39,Conde,BR,78.80,-7.26,-34.91,Pousada Beija Flor
44,Souillac,MU,75.20,-20.52,57.52,Bils' Villa
50,Capoeiras,BR,74.57,-8.73,-36.63,Restaurante & Hotel Deus e Fiel
51,Cap Malheureux,MU,75.00,-19.98,57.61,Pereybere Hotel & Spa


In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [21]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))